# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Fetch Dataset

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [4]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label == "2":
        x.append(example)
        y.append(-1)
    elif label == "7":
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
 0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
 0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608
 0.14092422 0.80219698 0.07455064 0.98688694 0.77224477 0.19871568
 0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
 0.35846573 0.11586906 0.86310343 0.62329813 0.33089802 0.06355835
 0.31098232 0.32518332 0.72960618 0.6375574

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4381625441696113
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.7385159010600706
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7785630153121319
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7714958775029446
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7868080094228505
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7703180212014135
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7620730270906949
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7573616018845701
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7361601884570083
test_accuracies: [0.7620730270906949, 0.7573616018845701, 0.7809187279151943, 0.7974087161366313, 0.8056537102473498, 0.806831566548881, 0.8032979976442873, 0.7597173144876326, 0.800942285041225, 0.8103651354534747, 0.7714958775029446, 0.7703180212014135, 0.7926972909305064, 0.7856301531213192, 0.7938751472320377, 0.8080094228504122, 0.7950530035335689, 0.7997644287396938, 0.7856301531213192, 0.7938751472320377, 0.7915194346289752, 0.8186101295641932, 0.7891637220259129, 0.8080094228504122, 0.7879858657243817, 0.7974087161366313, 0.7879858657243817, 0.7985865724381626, 0.7538280329799765, 0.7361601884570083]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.8091872791519434
test_accuracies: [0.7573616018845701, 0.7809187279151943, 0.7974087161366313, 0.8056537102473498, 0.806831566548881, 0.8032979

epoch 451
epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.8209658421672557
best accuracy so far!
test_accuracies: [0.7703180212014135, 0.7926972909305064, 0.7856301531213192, 0.7938751472320377, 0.8080094228504122, 0.7950530035335689, 0.7997644287396938, 0.7856301531213192, 0.7938751472320377, 0.7915194346289752, 0.8186101295641932, 0.7891637220259129, 0.8080094228504122, 0.7879858657243817, 0.7974087161366313, 0.7879858657243817, 0.7985865724381626, 0.7538280329799765, 0.7361601884570083, 0.8091872791519434, 0.7985865724381626, 0.7550058892815077, 0.7997644287396938, 0.7173144876325088, 0.8197879858657244, 0.7985865724381626, 0.7868080094228505, 0.7809187279151943, 0.7891637220259129, 0.8209658421672557]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.8445229681978799
best accuracy so far!
test_accuracies: [0.7926972909305064, 0.7856301531213192, 0.7938751472320377, 0.

epoch 561
epoch 562
epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.7809187279151943
test_accuracies: [0.7891637220259129, 0.8080094228504122, 0.7879858657243817, 0.7974087161366313, 0.7879858657243817, 0.7985865724381626, 0.7538280329799765, 0.7361601884570083, 0.8091872791519434, 0.7985865724381626, 0.7550058892815077, 0.7997644287396938, 0.7173144876325088, 0.8197879858657244, 0.7985865724381626, 0.7868080094228505, 0.7809187279151943, 0.7891637220259129, 0.8209658421672557, 0.8445229681978799, 0.8268551236749117, 0.8162544169611308, 0.7962308598351001, 0.7762073027090695, 0.8209658421672557, 0.8162544169611308, 0.7632508833922261, 0.7879858657243817, 0.7985865724381626, 0.7809187279151943]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7703180212014135
test_accuracies: [0.8080094228504122, 0.7879858657243817, 0.7974087161366313, 0.7879858657243817, 0.7985865724381626, 0.7538

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-0.03429426  1.16797461  0.19512243  1.77165922  1.1534362   0.4621478
 -0.21871851  0.82186079  1.29771369  1.25924991 -0.08161484  0.18992166
  1.31306646  1.0945027  -0.03747859  0.0510022  -0.05761669  1.07412194
  0.40430248  1.15075933  0.84183487  0.81850237  0.39396982  0.42761952
  2.17856077  0.42146576  1.16901945  0.50418544  0.60693008 -0.29328053
  1.48360601 -0.63370372 -0.48859684  0.59991613  0.91468111  0.28832731
  0.1326525  -0.70052267  0.95599065  0.33027425  1.59562682  0.10473484
  1.09656688  1.05332466  0.87729352  0.26551302  0.42300905  0.99399132
  0.58150303 -0.32766326  1.60589142  0.23036574  0.4062596   0.70816517
  0.50535601  0.84198153  0.81796375  0.45712603 -0.9256544   1.36569871
  0.52091328 -0.38834754  0.91579094 -0.13364389  0.04126787  0.08114164
  0.28339476  0.11287961  0.02116094  2.17316897  0.16381252  0.02093185
  0.0341671  -0.10079507  0.70685734  0.8802912   1.65115362  0.05427015
  1.12167252 -0.37707508 

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7886140923938404


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 53.911194801330566
training time: 17005.07194709778
test time: 576.7413275241852
total time: 17635.724469423294
